In [1]:
%load_ext lab_black

In [2]:
!rm -r data/box_effdet2/

In [3]:
import math
import os
import random
import shutil
import json
import logging
import sys
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm
from PIL import Image

import utils
import networks

In [4]:
MEAN = torch.tensor((0.485, 0.456, 0.406), dtype=torch.float32).reshape(3, 1, 1)
STD = torch.tensor((0.229, 0.224, 0.225), dtype=torch.float32).reshape(3, 1, 1)

PAD = 30
OUT_DIR = Path("data/box_effdet2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
class ImageDataset(Dataset):
    def __init__(self, files, keypoints=None, padding=40):
        super().__init__()
        self.files = files
        self.keypoints = keypoints
        self.padding = padding

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        f = self.files[idx]
        img = imageio.imread(f)
        x = img[38:1062, 374:1526]  # 1152x1024
        x = torch.tensor(x, dtype=torch.float32).permute(2, 0, 1).div(255.0)
        x = (x - MEAN) / STD

        return x

In [6]:
train_imgs = sorted(list(Path("data/ori/train_imgs/").glob("*.jpg")))
test_imgs = sorted(list(Path("data/ori/test_imgs/").glob("*.jpg")))

In [7]:
ds_train = ImageDataset(train_imgs)
ds_test = ImageDataset(test_imgs)

In [8]:
dl_train = DataLoader(ds_train, batch_size=20, num_workers=8, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=20, num_workers=8, shuffle=False)

In [9]:
model = networks.EfficientDet("efficientdet-d7", pretrained=True)
model = model.cuda()
model.eval()
torch.set_grad_enabled(False)
pass

In [10]:
dir_train = OUT_DIR / "train_imgs"
dir_test = OUT_DIR / "test_imgs"
dir_train_keypoint = OUT_DIR / "train_keypoints"
dir_train.mkdir(parents=True, exist_ok=True)
dir_test.mkdir(parents=True, exist_ok=True)
dir_train_keypoint.mkdir(parents=True, exist_ok=True)

In [11]:
shutil.copy("data/ori/train_df.csv", OUT_DIR / "train_df.csv")

PosixPath('data/box_effdet2/train_df.csv')

In [12]:
df = pd.read_csv(OUT_DIR / "train_df.csv")

In [13]:
torch.cuda.empty_cache()
i = 0
result_train = []
with tqdm(total=len(dl_train.dataset), ncols=100, file=sys.stdout) as t:
    for xs in dl_train:
        xs_ = xs.cuda()
        outs = model(xs_)
        for x, out in zip(xs, outs):
            t.set_postfix_str(train_imgs[i].name)
            x = ((x * STD + MEAN) * 255.0).permute(1, 2, 0).type(torch.uint8).numpy()

            # 사람 주변에 약간의 영역을 만들어줌
            h, w, _ = x.shape
            box = utils.get_single_person_rois(out)
            k = df.iloc[i, 1:].values.astype(np.float32)
            k = np.stack([k[0::2], k[1::2]], 1)
            x, k, offset, ratio = utils.resize_box(x, PAD, box, k)

            # 이미지 저장
            result_train.append({"image": train_imgs[i].name, "boxes": offset, "ratio": ratio})
            imageio.imwrite(dir_train / train_imgs[i].name, x)

            # 키포인트를 입힌 이미지도 모두 출력.
            # 키포인트가 제대로 잡혀있는지 확인
            keypoint_img = utils.draw_keypoints(x, k)
            imageio.imwrite(dir_train_keypoint / train_imgs[i].name, keypoint_img)

            i += 1
            t.update()

100%|████████████████████████████| 4195/4195 [08:38<00:00,  8.10it/s, 642-2-4-31-Z148_E-0000031.jpg]


In [14]:
torch.cuda.empty_cache()
i = 0
result_test = []
with tqdm(total=len(dl_test.dataset), ncols=100, file=sys.stdout) as t:
    for xs in dl_test:
        xs_ = xs.cuda()
        outs = model(xs_)
        for x, out in zip(xs, outs):
            t.set_postfix_str(train_imgs[i].name)
            x = ((x * STD + MEAN) * 255.0).permute(1, 2, 0).type(torch.uint8).numpy()

            # 사람 주변에 약간의 영역을 만들어줌
            h, w, _ = x.shape
            box = utils.get_single_person_rois(out)
            k = df.iloc[i, 1:].values.astype(np.float32)
            k = np.stack([k[0::2], k[1::2]], 1)
            x, k, offset, ratio = utils.resize_box(x, PAD, box, k)

            # 이미지 저장
            result_test.append({"image": test_imgs[i].name, "boxes": offset, "ratio": ratio})
            imageio.imwrite(dir_test / test_imgs[i].name, x)

            i += 1
            t.update()

100%|█████████████████████████████| 1600/1600 [03:04<00:00,  8.67it/s, 210-2-1-11-Z36_C-0000025.jpg]


In [16]:
with open(OUT_DIR / "offset.json", "w") as f:
    json.dump({"train": result_train, "test": result_test}, f)